In [38]:
import pandas as pd
import numpy as np
import networkx as nx
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
from collections import Counter, defaultdict

from tqdm import tqdm

In [24]:
# Labeling (getting type and nametag to dictionary)

add_label = pd.read_csv("~/Project/Ethereum/dataset/addresslabel.txt", names = ["addID", "type", "nametag"] , sep = " ")
add_label = add_label.set_index("addID")
label_dic = add_label.to_dict("index")

# miner and genesis list
genesis = pd.read_csv("~/Project/Ethereum/dataset/genesis_labels.txt", usecols = [0,1], names = ["addID", "type"] , sep = " ")
genesis = genesis.set_index("addID")
genesis_dic = genesis.to_dict("index")

miner = pd.read_csv("~/Project/Ethereum/dataset/blockminer.txt", names = ["blockID", "addID","uncle"], header=None, sep=' ')
miner_type = miner.drop_duplicates(["addID"])
miner_type["type"] = "miner" 
miner_type = miner_type[["addID", "type"]].set_index("addID")
miner_dic = miner_type.to_dict("index")

# setting multitype node
node_label_dic = defaultdict(lambda: defaultdict(list))

for node, type_dic in genesis_dic.items():
    node_label_dic[node]['type'].append('genesis')
for node, type_dic in miner_dic.items():
    node_label_dic[node]['type'].append('miner')

for node, type_dic in label_dic.items():
    if type_dic['type'] not in node_label_dic[node]['type']:
        node_label_dic[node]['type'].append(type_dic['type'])
    node_label_dic[node]['nametag'] = type_dic['nametag']

<ipython-input-24-121469dad2ad>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  miner_type["type"] = "miner"


In [27]:
# transaction=pd.read_csv("transaction_data.csv")
transaction=pd.read_csv("../preprocessed/transaction_data_except_txfee.csv")
# timestamp to datetime 
transaction["txdatetime"] = transaction["txtime"].map(lambda x:datetime.fromtimestamp(x, timezone.utc))

# delete self transaction 
transaction = transaction[transaction['in']!=transaction['out']] # excluding self transaction 

<ipython-input-27-9a4be306afbc>:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  timeseries_degree_centrality = pd.Series(index=date_idx)


In [51]:
monthly_network = []
date_idx = pd.date_range(start='08/01/2015', end='12/01/2016', freq='MS', tz='UTC') 

for time_from in tqdm(date_idx):    
    # aggregate with the time interval
    time_until = time_from + relativedelta(months=1)
    tx_bytime = transaction[(transaction.txdatetime >= time_from) & (transaction.txdatetime < time_until)] 
    real_tx = tx_bytime.groupby(["in", "out"], as_index=False).agg({"value": np.sum})
    real_tx = real_tx[real_tx["value"] != 0]
    real_tx["log_value"] = real_tx["value"].apply(lambda x: np.log10(x))
    
    tx_G_bytime = nx.from_pandas_edgelist(real_tx, source = "out", target = "in", edge_attr=["value"], create_using=nx.DiGraph())

    nx.set_node_attributes(tx_G_bytime, node_label_dic)

    # save_network 
    month = time_from.strftime('%Y%m')
    nx.write_gml(tx_G_bytime, f'/data01/storage_damini/Ethereum_network/monthly_network_{month}.gml')

100%|██████████| 17/17 [00:26<00:00,  1.55s/it]


### Monthly Network Analysis

In [ ]:
# degree distribution 
# weighted degree distribution

In [ ]:
# set one step
import powerlaw
import seaborn as sns
import statsmodels.api as sm
colors = ['r','g','b','c','m','y']
plt.figure(figsize = (12,9))

power_law_exponent=[]
monthly_network = []



distribution = tx_G_bytime.degree()
    distribution = tx_G_bytime.degree(weight='value')

    in_wdegree, out_wdegree = {}, {}
    for node in tx_G_bytime.nodes():
        in_wdegree[node] = 0
        out_wdegree[node] = 0


    for (in_, out_), value in nx.get_edge_attributes(tx_G_bytime, 'value').items():
        in_wdegree[in_] += value
        out_wdegree[out_] += value 

    for node, v in in_wdegree.items():
        in_wdegree[node] = np.log10(v)
    for node, v in out_wdegree.items():
        out_wdegree[node] = np.log10(v)


    alpha = power_law_fitting(tx_G_bytime.degree())
    # ain, aout = power_law_fitting(tx_G_bytime.in_degree()), power_law_fitting(tx_G_bytime.out_degree())
    gamma = power_law_fitting(distribution)
    # gin, gout = power_law_fitting(list(in_wdegree.items())), power_law_fitting(list(out_wdegree.items()))
    
    alpha = power_law_fitting(tx_G_bytime.degree())
    power_law_exponent.append([alpha])